# Chapter 10 - RANSAC & Bundle Adjustment

In this chapter we are looking into how we can construct a robust SFM. Since when working with SFM it can often happen that features are wrongly matched. If we determine the motion from the wrongly matched features we get totally wrong solution therefore we have to filter out such outliers first.



### Robust Estimation

When matching features there are usually some outliers, meaning point correspondences which are matched wrongly. Such outliers are caused by changing view points and changes is illumination. Also image noise, occlisions or blur can cause outliers. 

![Matching Example](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_10/1_matching_example.png)
*Figure 1: Matching Example. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*

When working with Motion estimation such faulty estimation can cause our estimation to drift far away from the truth since all these small deviation do add up.

![Outliers on Motion Estimation](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_10/2_outliers_on_motion_estimation.png)
*Figure 2: Outliers on Motion Estimation. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*

### RANdom SAmple Consensus (RANSAC)

A solution to having many outliers is the RANSAC method which is the standard method of fitting models in presence of having outliers. Since we have a normal fitting problem this solution can be applied whenever the goal is to estimate the parameters of a model. Therefore this is not exclusive to motion estimation and can allso be applied for example to SFM, Calibration or PnP and more.

The basic concept of RANSAC is that it samples randomly points and the tries to fit a model that describe these points. Then for all remaining points the error is calculated and the number of points inside e current region around our model/line are counted. Then the process is repeated. In the end we hselect the model with most points inside a certain region as the model.

So here is a example in more detail. Let go through the images from left to right. Given a set of points we select two points randomly. Then We fit a model throught these points. In this example we just draw a line. Then we calculate the error of all other points to that model. In our case this is just the shortest distance from our model/line to the point. Then we select all the data that support the hypothsis of our current model. This means we select the points that are inside a certain threshold around the model.

![Sampling and Fitting](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_10/3_random_sampling.png)
*Figure 3: Sampling and Fitting. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*

This process is denn repeated for many different random samples.

![Repeating](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_10/4_repeating_process.png)
*Figure 4: Repeating. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*

After k iterations we select the modle that hast most datapoint inside its support region. 

So how can we make sure that the produced results are reliable. As seen in the example when we only iterate 3 times and each time only select 2 points for fitting the model we will no reach a meaning full result. In the example above only at the forth iteration we reach a reasonable model. So what what is the optimal amount of point to use for fitting and what it the optmal number of iterations? 
So idelly we would always select 2 points for fitting and check all possible combinations of them. This however is computationallya very expensive since then there are $N(N-1)/2$ combinations possible for $N$ points. Even with only 1000 points this leadt to 500'000 combinations. So we would like to reduce the cost by not checking all possibilites and stop the RANSAC before. We can find out the optimal number of iterations in a probabilistic way.

#### Optimal k

First we have to define our prefered support range for this we have to set the inlierratio, so what ratio of all points have to be inside the support range. 

\begin{align*}
\textrm{inlier ratio} &= \textbf{w} := \textrm{number of inliers}/N \\
\textrm{number of points} &= \textbf{N}\\
\textrm{probaility of selecting inliers for both points} &= \mathbf{w^2} := P(p_1 == \textrm{inlier} \land p_2 == \textrm{inlier})\\
\textrm{probaility of selecting at least one outlier} &= \mathbf{1-w^2} := P(p_1 == \textrm{outlier} \lor p_2 == \textrm{outlier})\\
\textrm{RANSAC iteration} &= \textbf{k}\\
\textrm{probability that RANSAC never selcted both points as inlier} &= \mathbf{(1-w^2)^k} := (P(p_1 == \textrm{outlier} \lor p_2 == \textrm{outlier}))^2\\
\textrm{probaility of success} &= \textbf{p} := P(\textrm{both points inlier and therefore model reasonable})\\
\textrm{probaility of failure} &= \mathbf{1-p} := P(\textrm{never both points intlier, none of the models reasonable})
\end{align*}

\begin{align*}
&\implies 1-p = (1-w^2)^k\\
&\implies k = \dfrac{\log(1-p)}{\log(1-w^2)}\\
\end{align*}

So if we know the ratio of inliers $w$ and the probaility of success $p$ we can determine the number of iterations. Usually the user can choose these two parameters and therefor can decide the probaility of success and the inlier ratio. For example if we set the probaility of success ($p$) to 99% and the inlier ration $w$ to 50% we get $k=16$ iterations. This are significanlty fewer iterations than all combinations. Also notices that this derivation does not directly depend on the total number of points. Only $w$ depends on the total number of points.

Now we have derived a formula for getting the optimal number of iteration when we sample two points in each iteration. How does this change we we sample $s$ points in each iteration? Well the only thing that changes it that we don't have to take $w$ to the power of two but rather power $s$.

\begin{align*}
&\implies k = \dfrac{\log(1-p)}{\log(1-w^\mathbf{s})}
\end{align*}

### RANSAC in SFM

No we show how to use RANSAC for structure from motion. First we need to define model and error measures as well as the number of pointss to sample. Well for SFM the model is the essential / fundamental matrix. So in general the model is defined by the rotation $R$ and the translation $T$. As seen in chapter 9 we need at least 5 points to estimate the model, however more practical are 8 points for then using the 8-point algorithm. And we need to know which error measure we want to use.

So as already described in the general case we start with sampling the point, this time we sample 8 of them. then we model them using the 8-point algorithm to recieve the rotation and translation. We check how many point correspondences support this hypothesis for the transformation. Then we repeate the process for k iteration. In the end we select the essential matrix which encompasses the rotation and translation which has most point corresponences supporting it.

For the 8-point RANSAC case we need $k = \dfrac{\log(1-p)}{\log(1-w^\mathbf{8})}$ iterations. For a probaility $p=99%$, inlier-ration of $w=50%$ this results in 1177 iterations.

For now we only considered the inlier ratio, but of course we can also express all above in terms of outlier ratio. For this we only have to set the outlier ration $\epsilon = 1-w$.

When we plot the number of RANSAC-iterations again the outlier-ratio we see that with increasing outlier ratio the number of itertions needed rises rapidly. We also see that reducing the number of point to sample reduces the number of iterations. So therefore it would be beneficial to not use 8 points but 5. 

![Iteratons Needed](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_10/5_outlier_ratio_vs_iterations.png)
*Figure 5: Iteratons Needed. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*

Indeed 5 points are enought for a good estimation if we incluse some motion constraints.

### Planar Motion

For many SFM application we can define clearmotion constrainst. For example for cars we have only planar motion as the car itself can not move into the vertical direction. Or Trains we can constraint the motion by the turning radius.

We will now forcus on the constraint of planar motion. For such motions we can describe the transformation in a different format. The translation of the origin of the reference frame of the car is described in polar coordinates which consists of the angular coordinate $\varphi$ and the radial coordinate $\rho$. Additionaly the rotation of the vehicle-refference frame is described with the angle $\theta$.

![Planar Motion](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_10/6_planar_motion.png)
*Figure 6: Planar Motion. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*

Fro these definitions of rotation and translation we can construct the new essential matrix.

![Epipolar Constraint](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_10/7_planar_motion_epipolar.png)
![Epipolar Constraint2](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_10/7_planar_motion_epipolar2.png)
*Figure 7: Epipolar Constraint. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*

It is important to notice that $E$ only has two degrees of freedom. These are $\varphi$ and $\theta$. $\rho$ however is not a DoF since it is just a scale factor. As a result 2 correspondences are enought to find a solution.

So is it possible to use even less (than 2) corresponences? The surbrising answer is yes. This is when we use the exploit that wheeled vehicles underly the non-holonomic constraint meaning that they follow a circular motion.

### Planar and Circular Motion

For Circular motion we only have one degree of freedom. This is because the circular motion is around the Instantaneous Center of Rotation (ICR). The motion angle around the ICR is the same as the rotation angle of the vihicle around its onw refernece frame origin. Therefore the only degree of freedom is $\theta$. Therefore only 1 point correspondence is needed. Since less points are obviously not possible this is the smalles parametrization possible and therefore produces the most efficient algorithm with only one iteration.

![Circular Motion](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_10/8_circular_motion.png)
![Circular Motion2](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_10/8_circular_motion2.png)
*Figure 8: Circular Motion. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*

When applying the above notedd essential matrix for the circular motion to the correspondences $p_1$ and $p_2$ we get the following equation:

\begin{align*}
{p_2}^T E p_1 = 0 &\implies sin(\dfrac{\theta}{2}) (u_2 + u_1) + cos(\dfrac{\theta}{2}) (v_2 - v_1) = 0\\
&\implies \theta = -2 tan^{-1}(\dfrac{v_2 - v_1}{u_2 + u_1})\\
\end{align*}

the 1-Point RANSAC is ONLY used to find the inliers, the motion is then estimated from them in the usual 6 Degrees of freedom using triangulation.